In [8]:
import pandas as pd

In [9]:
# load the data
df = pd.read_csv('norris_et_al_2017_cisplatin_exp_data.csv.gz',
                 low_memory=False)

In [10]:
# get only the protein columns (assuming model only has proteins)
genes = df[df['species_type'] == 'protein'].copy()


In [11]:
# Create a list of genes to have in model
# This uses HGNC naming standard. 
# youll have to pass the right names from the model
genes_to_include = ['TP53', 'BBC3', 'BAX', 'BAK1', 'MDM2', 'BCL2L11', 'APAF1']

# Filter to get only these genes
select_genes = genes[genes['gene'].isin(genes_to_include)].copy()


In [12]:
# pivot tables to get into format where columns is the protein, rows are time points
pivoted = pd.pivot_table(select_genes, 
                         index ='time',
                         columns='protein', 
                         values='treated_control_fold_change')

# drop index
pivoted.reset_index(inplace=True)
del pivoted.columns.name
pivoted.set_index('time', inplace=True)
print(pivoted)

# note that there are multiple measurement types
# Phosphylation, rnaseq (rna level), silac and label-free(protein level)
# Decide based on model

      APAF1_rnaseq  BAK1_rnaseq  BAX_(ox)_38_57_phsilac  \
time                                                      
01hr     -1.290585     1.000404                     NaN   
06hr     -1.360740     1.344035                     NaN   
24hr      2.639418     4.275343                     NaN   
48hr           NaN          NaN                  1.3833   

      BAX_(ox)_38_58_phsilac  BAX_22_34_phsilac  BAX_N-term M(ace)1_lf  \
time                                                                     
01hr                     NaN                NaN                    NaN   
06hr                     NaN                NaN                   1.09   
24hr                     NaN                NaN                    NaN   
48hr                   1.322          -1.274372                    NaN   

      BAX_lf  BAX_rnaseq  BAX_silac  BBC3_rnaseq  BCL2L11_S(ph)109_phsilac  \
time                                                                         
01hr   -0.01    1.022159  -1.049432     1.03

In [25]:
# Rename the columns to match the PySB model

exp_data = pivoted.rename(
    columns={'TP53_S(ph)313_phsilac':'obs1'}
)


# Have to think of what to fill these with!
exp_data.fillna(1, inplace=True)

In [26]:
# create a new column with the phospho silac data averaged to be the active p53 concentration

exp_data['Obs_p53A'] = (exp_data['TP53_S(ph)392_phsilac'] + exp_data['TP53_S(ph)313_phsilac'])/2

print(exp_data)


      APAF1_rnaseq  BAK1_rnaseq  BAX_(ox)_38_57_phsilac  \
time                                                      
01hr     -1.290585     1.000404                  1.0000   
06hr     -1.360740     1.344035                  1.0000   
24hr      2.639418     4.275343                  1.0000   
48hr      1.000000     1.000000                  1.3833   

      BAX_(ox)_38_58_phsilac  BAX_22_34_phsilac  BAX_N-term M(ace)1_lf  \
time                                                                     
01hr                   1.000           1.000000                   1.00   
06hr                   1.000           1.000000                   1.09   
24hr                   1.000           1.000000                   1.00   
48hr                   1.322          -1.274372                   1.00   

      BAX_lf  BAX_rnaseq  BAX_silac  BBC3_rnaseq  BCL2L11_S(ph)109_phsilac  \
time                                                                         
01hr   -0.01    1.022159  -1.049432     1.03

In [ ]:
# rename the indices so that the names math the observables in the model

exp_data = pivoted.rename(
    index ={'Obs_p53A':'Obs_p53A'}
)

exp_data = pivoted.rename(
    index ={'MDM2_rnaseq':'Obs_mdm2'}
)

exp_data = pivoted.rename(
    index ={'BBC3_rnaseq':'Obs_Puma'}
)

In [20]:
def cost_function(trajectory):
    # can use any distance metric
    error1 = ((exp_data['obs1'] - trajectory['obs1'])**2).sum()
    
    # if there is more than one observable, you just repeat
    # error2 = ((exp_data['obs2'] - trajectory['obs2'])**2).sum()
    # return = error1 + error2
    return error1,

In [8]:
# demo data
# Dont need to worry about this, i just had to make data.
# this will be replaced with the simulation result from pysb.


x = [1, 2, 3, 4]
time = ['01hr', '06hr', '24hr', '48hr']
combines = [(i,j) for i,j in zip(time, x)]

traj = pd.DataFrame(combines, columns=['time', 'obs1'])
traj.set_index('time', inplace=True)

print(traj)

      obs1
time      
01hr     1
06hr     2
24hr     3
48hr     4


In [9]:
print(cost_function(traj))

(90.1927929272021,)


In [10]:
exp_data.to_csv('experimental_data.csv')